In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('15stocks_price.csv')

df['date_gsheets'] = pd.to_datetime(df['date_gsheets'])
df.set_index('date_gsheets', inplace=True)

In [16]:
returns_df = df.pct_change(1)
returns_df.dropna(inplace=True)

In [3]:
def getPortRisk(weights):
    returns_df = df.pct_change(1).dropna()
    num_stocks = len(returns_df.columns)
    vcv = returns_df.cov()
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))
    sd_p = np.sqrt(var_p)
    sd_p_annual = sd_p * np.sqrt(250)
    
    return sd_p_annual

In [4]:
num_stocks = len(df.columns)
init_weights = [1 / num_stocks] * num_stocks

In [5]:
bounds = tuple((0,1) for i in range(num_stocks))

In [6]:
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

In [7]:
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)

In [8]:
results

     fun: 0.1059347936948761
     jac: array([0.10597393, 0.10585235, 0.10600164, 0.1058007 , 0.10597585,
       0.10618784, 0.10613558, 0.10582459, 0.10587699, 0.11916338,
       0.11288869, 0.10579355, 0.10934387, 0.1058938 , 0.10634915])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([4.79798174e-02, 2.80742498e-01, 7.53106181e-03, 1.29294763e-01,
       3.54257633e-02, 8.40458002e-02, 1.62830960e-01, 1.72868281e-01,
       2.00872883e-02, 0.00000000e+00, 1.62630326e-18, 3.55672017e-03,
       2.49366500e-18, 4.31645324e-02, 1.24725144e-02])

In [9]:
getPortRisk(init_weights)

0.13290599867193698

In [10]:
optimized_weights = pd.DataFrame(results['x'])
optimized_weights.index = df.columns
optimized_weights.rename(columns={optimized_weights.columns[0] : 'weights'}, inplace=True)

In [11]:
optimized_weights

,weights
AAPL,4.797982e-02
KO,2.807425e-01
NFLX,7.531062e-03
BRK.B,1.292948e-01
DIS,3.542576e-02
IBM,8.404580e-02
VZ,1.628310e-01
WMT,1.728683e-01
GE,2.008729e-02
TSLA,0.000000e+00


In [12]:
optimized_weights['weights_rounded'] = optimized_weights['weights'].apply(lambda x : round(x, 3))

In [13]:
optimized_weights

,weights,weights_rounded
AAPL,4.797982e-02,0.048
KO,2.807425e-01,0.281
NFLX,7.531062e-03,0.008
BRK.B,1.292948e-01,0.129
DIS,3.542576e-02,0.035
IBM,8.404580e-02,0.084
VZ,1.628310e-01,0.163
WMT,1.728683e-01,0.173
GE,2.008729e-02,0.020
TSLA,0.000000e+00,0.000


In [14]:
optimized_weights['weights_rounded'].sort_values(ascending=False).cumsum()

KO       0.281
WMT      0.454
VZ       0.617
BRK.B    0.746
IBM      0.830
AAPL     0.878
UN       0.921
DIS      0.956
GE       0.976
V        0.988
NFLX     0.996
AMZN     1.000
TSLA     1.000
MA       1.000
MSFT     1.000
Name: weights_rounded, dtype: float64

In [17]:
corr_matrix = returns_df.corr().round(4)

In [18]:
corr_matrix

,AAPL,KO,NFLX,BRK.B,DIS,IBM,VZ,WMT,GE,TSLA,MA,AMZN,MSFT,UN,V
AAPL,1.0000,0.2054,0.1153,0.3138,0.2653,0.2638,0.1692,0.1729,0.2557,0.1971,0.3487,0.2587,0.3281,0.2541,0.2970
KO,0.2054,1.0000,0.0906,0.4723,0.3688,0.3207,0.3702,0.3104,0.3484,0.1439,0.3591,0.2338,0.3200,0.4058,0.3321
NFLX,0.1153,0.0906,1.0000,0.1852,0.1532,0.1237,0.0447,0.1006,0.1575,0.2317,0.2525,0.2955,0.2109,0.1504,0.2171
BRK.B,0.3138,0.4723,0.1852,1.0000,0.5385,0.4842,0.4160,0.3335,0.5484,0.2199,0.5331,0.3253,0.4262,0.4188,0.5030
DIS,0.2653,0.3688,0.1532,0.5385,1.0000,0.3432,0.3364,0.2736,0.4321,0.2277,0.4270,0.3083,0.3474,0.3292,0.4082
IBM,0.2638,0.3207,0.1237,0.4842,0.3432,1.0000,0.2970,0.2265,0.4255,0.1848,0.3839,0.2324,0.3876,0.3045,0.3562
VZ,0.1692,0.3702,0.0447,0.4160,0.3364,0.2970,1.0000,0.2922,0.3706,0.1104,0.2785,0.1688,0.2945,0.3091,0.2683
WMT,0.1729,0.3104,0.1006,0.3335,0.2736,0.2265,0.2922,1.0000,0.2426,0.1089,0.2484,0.1391,0.2242,0.2346,0.2470
GE,0.2557,0.3484,0.1575,0.5484,0.4321,0.4255,0.3706,0.2426,1.0000,0.1863,0.4010,0.2625,0.3353,0.3315,0.3753
TSLA,0.1971,0.1439,0.2317,0.2199,0.2277,0.1848,0.1104,0.1089,0.1863,1.0000,0.2716,0.2587,0.2108,0.1812,0.2250


In [19]:
corr_matrix[['NFLX', 'TSLA']]

,NFLX,TSLA
AAPL,0.1153,0.1971
KO,0.0906,0.1439
NFLX,1.0000,0.2317
BRK.B,0.1852,0.2199
DIS,0.1532,0.2277
IBM,0.1237,0.1848
VZ,0.0447,0.1104
WMT,0.1006,0.1089
GE,0.1575,0.1863
TSLA,0.2317,1.0000


In [20]:
corr_matrix[['AAPL', 'MA', 'MSFT']]

,AAPL,MA,MSFT
AAPL,1.0000,0.3487,0.3281
KO,0.2054,0.3591,0.3200
NFLX,0.1153,0.2525,0.2109
BRK.B,0.3138,0.5331,0.4262
DIS,0.2653,0.4270,0.3474
IBM,0.2638,0.3839,0.3876
VZ,0.1692,0.2785,0.2945
WMT,0.1729,0.2484,0.2242
GE,0.2557,0.4010,0.3353
TSLA,0.1971,0.2716,0.2108


In [21]:
corr_matrix[['KO', 'BRK.B']]

,KO,BRK.B
AAPL,0.2054,0.3138
KO,1.0000,0.4723
NFLX,0.0906,0.1852
BRK.B,0.4723,1.0000
DIS,0.3688,0.5385
IBM,0.3207,0.4842
VZ,0.3702,0.4160
WMT,0.3104,0.3335
GE,0.3484,0.5484
TSLA,0.1439,0.2199
